In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
file_path="data/quijote.txt"
with open(file_path, "r", encoding='utf-8') as fd:
    text = fd.read()

In [37]:
print(f"Lenght of the dataset in characters: {len(text)}")

Lenght of the dataset in characters: 2062750


In [38]:
print(text[0:1000])

Primera parte del ingenioso hidalgo don Quijote de la Mancha
Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha
En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. Una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda. El resto della concluían sayo de velarte, calzas de
velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de
entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una
ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte,
y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la
podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de
complexión recia, seco de carnes, enjut

In [39]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print("Characters set: " + "".join(chars))
print(f"Vocabulary size: {vocab_size}")

Characters set: 
 !"'(),-.01234567:;?ABCDEFGHIJLMNOPQRSTUVWXYZabcdefghijlmnopqrstuvxyz¡«»¿ÁÉÍÓÚáéíñóúü
Vocabulary size: 86


In [40]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Buenos días Sancho"))
print(decode(encode("Buenos días Sancho")))

[22, 65, 50, 58, 59, 63, 1, 49, 81, 46, 63, 1, 38, 46, 58, 48, 53, 59]
Buenos días Sancho


In [41]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([2062750]) torch.int64
tensor([35, 62, 54, 57, 50, 62, 46,  1, 60, 46, 62, 64, 50,  1, 49, 50, 56,  1,
        54, 58, 52, 50, 58, 54, 59, 63, 59,  1, 53, 54, 49, 46, 56, 52, 59,  1,
        49, 59, 58,  1, 36, 65, 54, 55, 59, 64, 50,  1, 49, 50,  1, 56, 46,  1,
        32, 46, 58, 48, 53, 46,  0, 23, 46, 60, 81, 64, 65, 56, 59,  1, 60, 62,
        54, 57, 50, 62, 59,  9,  1, 36, 65, 50,  1, 64, 62, 46, 64, 46,  1, 49,
        50,  1, 56, 46,  1, 48, 59, 58, 49, 54, 48, 54, 83, 58,  1, 68,  1, 50,
        55, 50, 62, 48, 54, 48, 54, 59,  1, 49, 50, 56,  1, 51, 46, 57, 59, 63,
        59,  1, 53, 54, 49, 46, 56, 52, 59,  0, 49, 59, 58,  1, 36, 65, 54, 55,
        59, 64, 50,  1, 49, 50,  1, 56, 46,  1, 32, 46, 58, 48, 53, 46,  0, 25,
        58,  1, 65, 58,  1, 56, 65, 52, 46, 62,  1, 49, 50,  1, 56, 46,  1, 32,
        46, 58, 48, 53, 46,  7,  1, 49, 50,  1, 48, 65, 68, 59,  1, 58, 59, 57,
        47, 62, 50,  1, 58, 59,  1, 61, 65, 54, 50, 62, 59,  1, 46, 48, 59, 62,
      

In [42]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [43]:
#context_lenght
block_size = 8
train_data[:block_size+1]

tensor([35, 62, 54, 57, 50, 62, 46,  1, 60])

In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([35]) the target: 62
when input is tensor([35, 62]) the target: 54
when input is tensor([35, 62, 54]) the target: 57
when input is tensor([35, 62, 54, 57]) the target: 50
when input is tensor([35, 62, 54, 57, 50]) the target: 62
when input is tensor([35, 62, 54, 57, 50, 62]) the target: 46
when input is tensor([35, 62, 54, 57, 50, 62, 46]) the target: 1
when input is tensor([35, 62, 54, 57, 50, 62, 46,  1]) the target: 60


In [45]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # Get rand ints between 0 and len(data) - block_size, with a 1d array batch_size
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[46, 57, 54, 63, 64, 46, 49,  7],
        [59,  1, 56, 50,  1, 56, 46, 66],
        [59,  9,  1, 25, 63,  7,  0, 60],
        [ 1, 50, 56,  0, 63, 50, 82, 59]])
targets:
torch.Size([4, 8])
tensor([[57, 54, 63, 64, 46, 49,  7,  1],
        [ 1, 56, 50,  1, 56, 46, 66, 46],
        [ 9,  1, 25, 63,  7,  0, 60, 65],
        [50, 56,  0, 63, 50, 82, 59, 62]])
----
when input is [46] the target: 57
when input is [46, 57] the target: 54
when input is [46, 57, 54] the target: 63
when input is [46, 57, 54, 63] the target: 64
when input is [46, 57, 54, 63, 64] the target: 46
when input is [46, 57, 54, 63, 64, 46] the target: 49
when input is [46, 57, 54, 63, 64, 46, 49] the target: 7
when input is [46, 57, 54, 63, 64, 46, 49, 7] the target: 1
when input is [59] the target: 1
when input is [59, 1] the target: 56
when input is [59, 1, 56] the target: 50
when input is [59, 1, 56, 50] the target: 1
when input is [59, 1, 56, 50, 1] the target: 56
when input is [59,

In [46]:
print(xb) # our input to the transformer

tensor([[46, 57, 54, 63, 64, 46, 49,  7],
        [59,  1, 56, 50,  1, 56, 46, 66],
        [59,  9,  1, 25, 63,  7,  0, 60],
        [ 1, 50, 56,  0, 63, 50, 82, 59]])


In [47]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 86])
tensor(5.0861, grad_fn=<NllLossBackward0>)

O6RJ»WaíéÉ'QGqzR
CÉpVXs)6ÓóÓ »¡nzSW¡púÉbA
iA)¿tfIJbIÓ¡¿Súóo!ú qÁEA6CTGíEuAgW4V.s»óHc,RvüEIVIQh2YUüIc


In [48]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [49]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.1726417541503906


In [50]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


dero-de y lodors, o, spejosíareterdal ue amasa denteco mo, mese hamos ba vielerder dicilles
delo e n abrecoidi ancido as bide qu brqun prábimpo Y di destracieso
cin.
-AÓgui to vas, amenta ias co trtes
níandala cuésilantrí ndres fue, l prmi quede que babren daza, Cuaseraren Esaseña marabía qucuso, vestele daue po ab(úbijuequan an e coprincie sijos, pome qusco pa de ha derdarr? Queñondes Ma güEs lia


## The mathematical trick in self-attention

In [72]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [73]:
# consider the following toy example:
torch.manual_seed(42)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [74]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [75]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [76]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [77]:
# version 4: self-attention!
torch.manual_seed(42)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [78]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1905, 0.8095, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3742, 0.0568, 0.5690, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1288, 0.3380, 0.1376, 0.3956, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4311, 0.0841, 0.0582, 0.3049, 0.1217, 0.0000, 0.0000, 0.0000],
        [0.0537, 0.3205, 0.0694, 0.2404, 0.2568, 0.0592, 0.0000, 0.0000],
        [0.3396, 0.0149, 0.5165, 0.0180, 0.0658, 0.0080, 0.0373, 0.0000],
        [0.0165, 0.0375, 0.0144, 0.1120, 0.0332, 0.4069, 0.3136, 0.0660]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [79]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [80]:
k.var()

tensor(1.0392)

In [81]:
q.var()

tensor(0.9791)

In [82]:
wei.var()

tensor(0.7791)

In [83]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [84]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [85]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [86]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [87]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

French to English translation example:

<--------- ENCODE ------------------><--------------- DECODE ----------------->

les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>


